In [1]:
from pymongo import MongoClient
import pandas as pd
import folium

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
companies=list(coll.find())

df=pd.DataFrame(companies)

In [5]:
print(set(df.category_code))

{'education', 'local', 'security', 'health', 'other', 'enterprise', 'cleantech', 'nanotech', 'web', 'transportation', 'public_relations', 'advertising', 'semiconductor', 'fashion', 'mobile', 'medical', 'automotive', 'social', 'real_estate', 'design', 'consulting', 'legal', 'hospitality', 'hardware', 'network_hosting', 'software', 'finance', 'search', 'analytics', 'manufacturing', 'games_video', 'nonprofit', 'travel', 'biotech', 'music', None, 'sports', 'photo_video', 'messaging', 'ecommerce', 'news', 'government'}


In [6]:
tech_companies=list(coll.find({"$and" : [{"$or":[{"category_code": 'web'},{'category_code':'analytics'},
                                                 {"category_code": 'software'},{"category_code": 'hardware'}]}, 
                                         {"offices.city": "New York"}]}))
companies_df=pd.DataFrame(tech_companies)

In [7]:
def getLocation(comp):
    longitude = comp['offices'][0]['longitude']
    latitude = comp['offices'][0]['latitude']
    loc = {
        'type':'Point',
        'coordinates':[latitude, longitude]
    }
    return loc

In [8]:
for e in tech_companies:
    value={"$set" : {"loc": getLocation(e)}}
    coll.update_one(e,value)

In [9]:
db, coll = connectCollection('companies','companies')

In [10]:
companies_df=pd.DataFrame(tech_companies)

print(companies_df['loc'])

companies_df['loc']

0      {'type': 'Point', 'coordinates': [47.603122, -...
1      {'type': 'Point', 'coordinates': [40.757725, -...
2      {'type': 'Point', 'coordinates': [40.72604, -7...
3      {'type': 'Point', 'coordinates': [37.393679, -...
4      {'type': 'Point', 'coordinates': [40.736416, -...
                             ...                        
296    {'type': 'Point', 'coordinates': [40.751804, -...
297       {'type': 'Point', 'coordinates': [None, None]}
298    {'type': 'Point', 'coordinates': [40.72616, -7...
299    {'type': 'Point', 'coordinates': [40.72616, -7...
300       {'type': 'Point', 'coordinates': [None, None]}
Name: loc, Length: 301, dtype: object


0      {'type': 'Point', 'coordinates': [47.603122, -...
1      {'type': 'Point', 'coordinates': [40.757725, -...
2      {'type': 'Point', 'coordinates': [40.72604, -7...
3      {'type': 'Point', 'coordinates': [37.393679, -...
4      {'type': 'Point', 'coordinates': [40.736416, -...
                             ...                        
296    {'type': 'Point', 'coordinates': [40.751804, -...
297       {'type': 'Point', 'coordinates': [None, None]}
298    {'type': 'Point', 'coordinates': [40.72616, -7...
299    {'type': 'Point', 'coordinates': [40.72616, -7...
300       {'type': 'Point', 'coordinates': [None, None]}
Name: loc, Length: 301, dtype: object

In [11]:
companies_df=companies_df[['name','loc']]

companies_df

,name,loc
0,Wetpaint,"{'type': 'Point', 'coordinates': [47.603122, -..."
1,Viacom,"{'type': 'Point', 'coordinates': [40.757725, -..."
2,Meetup,"{'type': 'Point', 'coordinates': [40.72604, -7..."
3,Meebo,"{'type': 'Point', 'coordinates': [37.393679, -..."
4,Forbes,"{'type': 'Point', 'coordinates': [40.736416, -..."
...,...,...
296,Datum Legal,"{'type': 'Point', 'coordinates': [40.751804, -..."
297,MedTrackAlert,"{'type': 'Point', 'coordinates': [None, None]}"
298,Vertro,"{'type': 'Point', 'coordinates': [40.72616, -7..."
299,ALOT,"{'type': 'Point', 'coordinates': [40.72616, -7..."


In [12]:
for i in range(len(companies_df['loc'])):
    if companies_df.loc[i]['loc']==[None,None]:
        print(True)
        break

In [13]:
companies_df=companies_df.loc[companies_df['loc']!={'type': 'Point', 'coordinates': [None, None]}]

companies_df

,name,loc
0,Wetpaint,"{'type': 'Point', 'coordinates': [47.603122, -..."
1,Viacom,"{'type': 'Point', 'coordinates': [40.757725, -..."
2,Meetup,"{'type': 'Point', 'coordinates': [40.72604, -7..."
3,Meebo,"{'type': 'Point', 'coordinates': [37.393679, -..."
4,Forbes,"{'type': 'Point', 'coordinates': [40.736416, -..."
...,...,...
294,Sword,"{'type': 'Point', 'coordinates': [40.7029757, ..."
295,smartFOCUS,"{'type': 'Point', 'coordinates': [51.4525116, ..."
296,Datum Legal,"{'type': 'Point', 'coordinates': [40.751804, -..."
298,Vertro,"{'type': 'Point', 'coordinates': [40.72616, -7..."


In [14]:
#para crear nuevas collections
myclient = MongoClient("mongodb://localhost/")
mydb = myclient["companies"]
mycol = mydb["Finding_best_place"]
#para importar el contenido de un csv a mongo
#mongoimport --db companies --collection finding_location --type csv --file Companies.csv --headerline

In [15]:
companies_df=companies_df.loc[:100]

In [21]:
companies_df['Category']='Company'

companies_df=companies_df[['name','loc','Category']]

companies_df[['name','loc','Category']].to_json('Output/companies.json',orient='records')

In [17]:
for i in companies_df.index:
    if type(companies_df.loc[i]['loc']['coordinates'][0])!=float:
        print(companies_df.loc[i])

In [18]:
companies_df.index

Int64Index([  0,   1,   2,   3,   4,   5,   7,   8,   9,  10,  11,  12,  13,
             14,  15,  16,  17,  18,  19,  22,  23,  24,  25,  26,  27,  28,
             32,  33,  34,  35,  36,  37,  38,  39,  40,  41,  43,  44,  45,
             46,  47,  48,  49,  50,  51,  52,  53,  54,  55,  57,  58,  59,
             60,  61,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  74,
             75,  76,  78,  80,  81,  82,  83,  85,  87,  89,  90,  92,  93,
             94,  95,  96,  97,  98,  99, 100],
           dtype='int64')

In [19]:
map_city = folium.Map(location=(40.7142700, -74.0059700), zoom_start=12)
map_city

In [20]:
for e in coordinates_comp:
    folium.Marker(e, radius=2,
                    icon=folium.Icon(prefix='fa',icon='cloud',color='red'), 
                   ).add_to(map_city)

NameError: name 'coordinates_comp' is not defined

In [ ]:
map_city

In [ ]:
companies_df[:12]